In [21]:
import osmnx as ox
import networkx as nx
import folium

In [28]:
# Define the individual places in Bicol Region
places = ["Masbate, Philippines", "Bicol Region, Philippines", "Catanduanes, Philippines"]

# Create an empty directed multigraph to combine all places
combined_graph = nx.MultiDiGraph()

# Loop through each place and retrieve the graph for each
for place in places:
    graph = ox.graph_from_place(place, custom_filter='["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified|ferry"]', network_type='drive')
    combined_graph = nx.compose(combined_graph, graph)

# Get the edges and nodes GeoDataFrames
gdf_nodes, gdf_edges = ox.graph_to_gdfs(combined_graph, nodes=True, edges=True)

# Get the centroid of the combined places to initialize the map
centroid = ox.geocode_to_gdf(places[0]).unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=8, tiles='OpenStreetMap')

# Add the edges (roads and sea routes) to the map
for _, edge in gdf_edges.iterrows():
    points = [(y, x) for x, y in edge['geometry'].coords]
    folium.PolyLine(points, color="darkred", weight=1.5, opacity=0.6).add_to(m)

# Add the nodes (intersections) to the map
for _, node in gdf_nodes.iterrows():
    folium.CircleMarker(location=[node['y'], node['x']],
                        radius=3, color='blue', fill=True, fill_opacity=0.8).add_to(m)

# Save the map
m.save("network.html")

C:\Users\ADMIN\miniconda3\envs\ox\Lib\site-packages\osmnx\_overpass.py:254: UserWarning: This area is 25 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
